In [1]:
import cv2
import os
import math
import mmcv
import pandas as pd
import torch
from time import sleep
import numpy as np
from matplotlib import pyplot as plt
from ultralytics import YOLO
from ultralytics.yolo.utils.plotting import Annotator

from mmpose.apis import inference_top_down_pose_model, init_pose_model, vis_pose_result
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


%run utils/superimposition/functions.ipynb
%run utils/extraction/objClass.ipynb
%run utils/skeleton/detect_skeletons.ipynb

C:\Users\gmald\anaconda3\envs\senior\lib\site-packages\mmcv\__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


In [2]:
class Configuration:
    def __init__(self, minimalImg, mainImg, segModel, BBModel, videoFormat, FPS, fileName, skip):
        self.minimalImg = minimalImg
        self.mainImg = mainImg
        
        segPath = "utils/segmentation/"
        
        self.segWeights = segPath+segModel
        self.BBWeights = segPath+BBModel
        
        self.fourcc = cv2.VideoWriter_fourcc(*videoFormat)
        
        self.FPS = FPS
        
        self.mainPath = f"videos/{fileName}/"
        self.processedPath = self.mainPath+"processed/"
        
        self.skip = skip
        
        self.pose_config = "utils/skeleton/demo/hrnet_w32_coco_256x192.py"
        self.pose_checkpoint = "https://download.openmmlab.com/mmpose/top_down/hrnet/hrnet_w32_coco_256x192-c78dce93_20200708.pth"
        
fileList = os.listdir("videos")
fileName = fileList[2]

print(fileList, "\nSelected File:", fileName)

config = Configuration(minimalImg="Thermal", mainImg="RGB", segModel="yolov7-seg.pt", BBModel="yolov7-tiny.pt", videoFormat="mp4v", FPS=10, fileName=fileName, skip=190 )

['2022-12-01_18-03-33', '2022-12-02_13-05-04', '2023-01-24_15-28-00', '2023-01-28_15-50-17', 'combined_outisde_13.gif', 'newStereo', 'outside_test', 'SII'] 
Selected File: 2023-01-24_15-28-00


In [3]:
seg_model = YOLO("yolov8n-seg.pt")
pose_model = init_pose_model(config.pose_config, config.pose_checkpoint, device)

Handler = ObjectHandler()
images = {"RGB": None, "Thermal": None, "Depth": None}

PATH = f"{config.processedPath}{config.mainImg}_sii.mp4"

rgbVideo = cv2.VideoCapture(f"{config.processedPath}RGB_sii.mp4")
thermalVideo = cv2.VideoCapture(f"{config.processedPath}Thermal_sii.mp4")
depthVideo = cv2.VideoCapture(f"{config.processedPath}Depth_sii.mp4")

trackout = cv2.VideoWriter(f"{config.processedPath}track.mp4", config.fourcc, config.FPS, (512,512))

frameCount = 0
for result in seg_model.track(source=PATH, stream=True, verbose=False, classes=[0,2], device=0):
    
    ret, images["RGB"] = rgbVideo.read()
    ret1, images["Thermal"] = thermalVideo.read()
    ret2, images["Depth"] = depthVideo.read()
    if not (ret & ret1 & ret2):
        break
        
    
    StereoframeMapped = cv2.normalize(images["Depth"], None, 255, 0, cv2.NORM_INF, cv2.CV_32F)
    StereoframeMapped = cv2.convertScaleAbs(StereoframeMapped)
    StereoframeMapped = cv2.cvtColor(StereoframeMapped, cv2.COLOR_BGR2GRAY)
    StereoframeMapped = cv2.equalizeHist(StereoframeMapped)
    StereoframeMapped = cv2.applyColorMap(StereoframeMapped, cv2.COLORMAP_JET)
    images["Color_Depth"] = StereoframeMapped
    
    
    if result.boxes and type(result.boxes.id) != type(None):
        
        # Collect Properties of Objects
        classes = result.boxes.cls
        masks = result.masks.data.cpu()
        boxes_array = np.array(result.boxes.xyxy.cpu())
        track_ids = result.boxes.id.cpu()
        frame_num = np.ones_like(classes) * frameCount
        object_results = [{'class': cls.item(),  'track_id': track_id.item(), 'bbox': bbox,  'mask': mask, 'frame_num': frame} 
                          for cls, track_id, bbox, mask, frame in zip(classes, track_ids, boxes_array, masks, frame_num)]
        
        images["Superimposed"] = apply_mask(images.copy(), masks, config.mainImg)
        
        object_results = inference_top_down_pose_model(pose_model, images[config.mainImg], object_results, format='xyxy')[0]
        
        Handler.appendObjects(images, object_results)
        images["Superimposed"] = vis_pose_result(pose_model, images["Superimposed"], object_results, bbox_color=(0,0,0))
    
        annotator = Annotator(images["Superimposed"])

        for obj in object_results:

            color = Handler.object_container[obj["track_id"]].color_id
            
            
            annotator.box_label(obj["bbox"], label=f'{obj["track_id"]}', color=color)
    
    frame = annotator.result()
    
    for label in images:
        cv2.imshow(label, images[label])
    
    cv2.imshow("YOLO", frame)
    
    trackout.write(images["Superimposed"])
    
    frameCount+=1
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

trackout.release()

load checkpoint from http path: https://download.openmmlab.com/mmpose/top_down/hrnet/hrnet_w32_coco_256x192-c78dce93_20200708.pth


In [4]:
_=Handler.createDataframe()

,frame_num,track_id,class,bbox,color,temperature,spatials,keypoints
0,0.0,1,0,"[182.50389, 222.00867, 216.63762, 330.8199]","[6, 2, 0]",70.749798,"[-1.4470411539646868, -0.5234926974657135, 9.0...","[[209.66556, 231.25336, 0.7894298], [209.66556..."
1,0.0,2,0,"[39.979027, 221.66737, 65.53487, 300.42285]","[6, 5, 1]",60.338636,"[-8.19005841147257, -0.20330192733994118, 14.2...","[[56.98698, 228.35852, 0.6702892], [55.448788,..."
2,0.0,3,0,"[494.0433, 224.4111, 511.827, 280.3014]","[18, 26, 53]",94.372960,"[4.029627597555326, 0.0594608254326273, 5.7647...","[[498.84164, 232.43442, 0.84143305], [499.3874..."
3,0.0,4,0,"[431.9463, 223.09326, 450.54474, 275.00488]","[68, 88, 108]",92.652116,"[15.114705920896274, 0.56714587231576, 28.8235...","[[440.4851, 229.02457, 0.7544142], [441.49896,..."
4,0.0,5,0,"[93.43863, 232.28745, 115.77039, 303.86487]","[0, 0, 0]",70.993255,"[-4.235241204558135, -0.33782710340719735, 9.8...","[[111.94399, 238.36876, 0.77869993], [111.9439..."
...,...,...,...,...,...,...,...,...
1266,149.0,14,0,"[223.1323, 227.58185, 253.8164, 309.92078]","[12, 11, 8]",87.548459,"[-0.6770457901428159, -0.49260496869443327, 13...","[[234.05185, 235.38153, 0.91945124], [236.4641..."
1267,149.0,16,0,"[-4.3005, 213.13919, 57.217823, 366.07117]","[36, 27, 17]",69.047023,"[-3.822230124323154, -0.5595796804351534, 5.88...","[[51.10102, 230.61285, 0.85076237], [51.10102,..."
1268,149.0,18,0,"[190.7612, 230.05212, 225.62556, 307.98386]","[45, 44, 38]",82.639487,"[-1.4010593724273814, -0.3815162462784477, 10....","[[210.09602, 237.43433, 0.9147525], [211.61812..."
1269,149.0,21,0,"[478.35413, 225.38406, 508.1836, 294.67746]","[18, 17, 13]",99.823650,"[4.108941942381202, -0.06980336945800579, 6.11...","[[490.90042, 231.94797, 0.86402494], [492.2538..."


In [6]:
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt

obj_spatials = np.array(Handler.object_container[18].properties['spatials']).transpose(1,0)

ax = plt.figure().add_subplot(projection='3d')

x = obj_spatials[0]
y = obj_spatials[1]
z = obj_spatials[2]

ax.plot(x, y, z, label='parametric curve')

ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')

ax.legend()

plt.show()